In [ ]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
from eofs.standard import Eof
import matplotlib.pyplot as plt
import IPython.display
import time
from scipy import signal

#### Define all functions we use later on

In [ ]:
def trans_linear_to_2D(data, side_length = 9):
    """Transforms an array of the shape (timesteps, number_grid_p) into (timesteps, sqrt(number_grid_p),sqrt(number_grid_p))"""
    t_length, linear_length = data.shape
    assert linear_length == side_length**2
    
    reshaped_data = np.reshape(data,(t_length,side_length,side_length), order = 'F')
    return reshaped_data

def trans_2D_into_linear(data):
    """Transforms an array of the shape (timesteps, sqrt(number_grid_p),sqrt(number_grid_p)) into (timesteps, number_grid_p)"""
    t_length, side_length, side_length_y= data.shape
    assert side_length == side_length_y
    reshaped_data = np.reshape(data,(t_length,side_length**2),order = 'F')
    
    return reshaped_data
    

In [ ]:
def reconstruct(eofs, pcs):
    """Function that reconstructs Fields from EOF and PC"""
    time_length, spatial_length = pcs.shape
    reconstructed_field = np.zeros(pcs.shape)
    
    for i in range(time_length):
        reconstructed_field[i,:] = np.sum(eofs*pcs[i,:,None], axis =0)
    return reconstructed_field

In [ ]:
def get_eofs(eof_rawdata):
    """Do EOF analysis and give out Eof, PC and corresponding variance"""
    solver = Eof(eof_rawdata,weights=None)
    eofs = solver.eofs()
    variance_fraction = solver.varianceFraction()
    pcs = solver.pcs()
    pseudo_pcs = solver.projectField(eof_rawdata)
    return eofs, pseudo_pcs, variance_fraction


In [ ]:
def pc_poly_extrapolation(pcs, polydegree, times, newtimes):
    """Polynomial extrapolation of pcs function"""
    pcs_extra = []
    diagnostics = []
    # polyft for pcs
    for i, pc in enumerate(pcs):
        output = np.polyfit(times, pc, polydegree, full=True)
        coefficients = output[0]
        pc_fitted = np.polyval(coefficients, newtimes)
        pcs_extra.append(pc_fitted)
        diagnostics.append(output[1:]) 
    return pcs_extra, diagnostics


def poly_residuals(pcs, pcs_fitted):
    residuals_arrays = []
    for pcs, pcsfit in zip(pcs, pcs_fitted):
        res = pcsfit[:len(pcs)] - pcs
        residuals_arrays.append(res)

    return residuals_arrays

def autocorrelations(residuals):
    autocorrs = []
    for res in residuals:
        result = np.correlate(np.array(res), np.array(res), mode='full')
        length = math.ceil(len(result)/2)
        autocorrelation = result[:length]
        autocorrs.append(autocorrelation)
    return autocorrs

#### Read data

In [ ]:
v_x_pandas=pd.read_csv('velocity_x.csv')
v_y_pandas=pd.read_csv('velocity_y.csv')

In [ ]:
v_x=v_x_pandas.values[:,1:]
v_y=v_y_pandas.values[:,1:]

In [ ]:
v_x_2D = trans_linear_to_2D(v_x)
v_y_2D = trans_linear_to_2D(v_y)

In [ ]:
alltimes = np.arange(v_x.shape[0])

#### Plot video of velocity field time development

In [ ]:
tstart = 3000
tstop = 3100
tstep = 50
var = v_y_2D
# defines field and time vector
trange = slice(tstart, tstop + 1, tstep)
tsteps = alltimes[trange]
field=var[trange, :, :]
# first plot
vmax = np.nanmax(field)
vmin = np.nanmin(field)
levels = np.linspace(vmin, vmax, 256)
fig, ax = plt.subplots(dpi=100)
cs = ax.contourf(field[0])
cbar = plt.colorbar(cs)
# display video
for i, f in enumerate(field):
    ax.collections = []
    cs = ax.contourf(f)
    plt.title("t = %s" % (tsteps[i]))    
    IPython.display.display(fig)
    IPython.display.clear_output(wait=True)
    time.sleep(0.5)

**U and V Seperated analysed in 1D**

In [ ]:
eofs_x, pcs_x, variance_x = get_eofs(v_x)
eofs_y, pcs_y, variance_y = get_eofs(v_y)

**2D Analysis**, is not necessary, we demonstrate, that it does not matter if we do it in 2D or 1D

In [ ]:
eofs_x_2D, pcs_x_2D, variance_x_2D = get_eofs(v_x_2D)

In [ ]:
plt.plot(variance_x_2D)
plt.plot(variance_x)

In [ ]:
trans_eofs_x_2D = trans_2D_into_linear(eofs_x_2D)

In [ ]:
plt.plot(eofs_x[0,:])
plt.plot(trans_eofs_x_2D[0,:])

**Test**, Can perfectly reconstruct field, and EoFs are orthogonal

In [ ]:
reconstructed_field_fun = reconstruct(eofs_x, pcs_x)
t_step = 100

In [ ]:
plt.plot(v_x[t_step,:], label = "act. field")
plt.plot(reconstructed_field_fun[t_step,:], label = "My rec")
plt.legend()

In [ ]:
#Test of Orthogonality
for j in range(81):
    for i in range(81):
        diag_sum = np.sum(eofs_x[j,:]*eofs_x[i,:])
        if diag_sum>1e-14:
            print(diag_sum, i, j)

#### Polyfit extrapolation of weights of pcs

In [ ]:
datastart = -200
datastop = None
extratime = 101
pcs_for_fit = slice(0, 81)
polydegree = 3

datapoints_for_fit = slice(datastart, datastop)
times = alltimes[datapoints_for_fit]
newtimes = np.arange(times[0], times[-1] + extratime)

In [ ]:
pcs_x_forfit = np.transpose(pcs_x[datapoints_for_fit, pcs_for_fit])
pcs_y_forfit = np.transpose(pcs_y[datapoints_for_fit, pcs_for_fit])

pcs_x_extra, diagnostics_x = pc_poly_extrapolation(pcs_x_forfit, polydegree, times, newtimes)
pcs_y_extra, diagnostics_y = pc_poly_extrapolation(pcs_y_forfit, polydegree, times, newtimes)

# diagnostics: residuals, rank, singular values, conditioning threshold
residuals_x = [dia[0] for dia in diagnostics_x]
mse_x = [list(res).pop() for res in residuals_x]
residuals_y = [dia[0] for dia in diagnostics_y]
mse_y = [list(res).pop() for res in residuals_y]
# print(residuals_x, residuals_y) # sum of the squares of the fit errors

#### Test the residuals of polyfit

In [ ]:
residuals_x_arrays = poly_residuals(pcs_x_forfit, pcs_x_extra)
residuals_y_arrays = poly_residuals(pcs_y_forfit, pcs_y_extra)

In [ ]:
autocorrs_x = autocorrelations(residuals_x_arrays)
autocorrs_y = autocorrelations(residuals_y_arrays)

#### Plot weights

In [ ]:
# plot only first x pcs
showplots = slice(0, 5)
i = 1
for pc, pc_extra, residuals, autocorr, pcleft in zip(pcs_x_forfit[showplots], pcs_x_extra[showplots], 
                                   residuals_x_arrays[showplots], autocorrs_x[showplots], np.transpose(pcs_x[:, showplots])):
    fig = plt.figure(dpi=100)
    plt.plot(newtimes, pc_extra)
    plt.plot(alltimes[datastart:], pcleft[datastart:])
    plt.plot(times, pc)
    plt.title("Extrapolation of weights: pc %s" %i)
    fig = plt.figure(dpi=100)
    plt.plot(times, residuals)
    plt.title("Residuals: pc %s" %i)
    fig = plt.figure(dpi=100)
    plt.hist(residuals)
    plt.title("Residuals: pc %s" %i)
    fig = plt.figure(dpi=100)
#     plt.plot(autocorr)
    plt.xcorr(residuals, residuals, maxlags=100)
    plt.title("Autocorrelation: pc %s" %i)
    i += 1

**Prediction**

In [ ]:
# prediction time steps we pick 
prediction_times = [25, 50, 75]
pcs_prediction_x = []
pcs_prediction_y = []
for pcx, pcy in zip(pcs_x_extra, pcs_y_extra):
    pcs_prediction_x.append(pcx[prediction_times])
    pcs_prediction_y.append(pcy[prediction_times])
pcs_prediction_x = np.transpose(pcs_prediction_x)
pcs_prediction_y = np.transpose(pcs_prediction_y)

#### Variance

In [ ]:
var_xplus = np.array([pred + mse_x for pred in pcs_prediction_x])
var_xminus = np.array([pred - mse_x for pred in pcs_prediction_x])
var_yplus = np.array([pred + mse_y for pred in pcs_prediction_y])
var_yminus = np.array([pred - mse_y for pred in pcs_prediction_y])

In [ ]:
mse_sum_x = np.sqrt(np.sum(np.array(mse_x)**2))
mse_sum_y = np.sqrt(np.sum(np.array(mse_y)**2))
print(mse_sum_x)
print(mse_sum_y)

#### Reconstruct predicted field

In [ ]:
predicted_x = reconstruct(eofs_x, pcs_prediction_x)
predicted_y = reconstruct(eofs_y, pcs_prediction_y)

In [ ]:
predicted_2D_x = trans_linear_to_2D(predicted_x)
predicted_2D_y = trans_linear_to_2D(predicted_y)

#### Reconstruct with variance

In [ ]:
predicted_var_xplus = reconstruct(eofs_x, var_xplus)
predicted_var_xminus = reconstruct(eofs_x, var_xminus)
predicted_var_yplus = reconstruct(eofs_y, var_yplus)
predicted_var_yminus = reconstruct(eofs_y, var_yminus)

In [ ]:
predicted_2D_var_xplus = trans_linear_to_2D(predicted_var_xplus)
predicted_2D_var_xminus = trans_linear_to_2D(predicted_var_xminus)
predicted_2D_var_yplus = trans_linear_to_2D(predicted_var_yplus)
predicted_2D_var_yminus = trans_linear_to_2D(predicted_var_yminus)

#### Last time step data

In [ ]:
fig = plt.figure(figsize=(8, 3), dpi=200)
fig.add_subplot(1,2,1)
plt.contourf(v_x_2D[-1, :, :])
plt.colorbar()
plt.axis("scaled")
plt.title("Velocity u")
fig.add_subplot(1,2,2)
plt.contourf(v_y_2D[-1, :, :])
plt.title("Velocity v")
plt.colorbar()
plt.axis("scaled")
plt.suptitle("Last data time step")
plt.tight_layout()
plt.subplots_adjust(top=0.8)

#### Prediction time steps

In [ ]:
for i, predtime in enumerate(prediction_times):
    fig = plt.figure(figsize=(8, 3), dpi=200)
    fig.add_subplot(1,2,1)
    plt.contourf(predicted_2D_x[i, :, :])
    plt.colorbar()
    plt.axis("scaled")
    plt.title("Velocity u")
    fig.add_subplot(1,2,2)
    plt.contourf(predicted_2D_y[i, :, :])
    plt.title("Velocity v")
    plt.colorbar()
    plt.axis("scaled")
    plt.suptitle("Prediction at time step %s" %predtime)
    plt.tight_layout()
    plt.subplots_adjust(top=0.8)

#### Variance of prediction time steps

In [ ]:
for i, predtime in enumerate(prediction_times):
    fig = plt.figure(figsize=(8, 3), dpi=200)
    fig.add_subplot(1,2,1)
    plt.contourf(predicted_2D_var_xminus[i, :, :])
    plt.colorbar()
    plt.axis("scaled")
    plt.title("Velocity u - MSE")
    fig.add_subplot(1,2,2)
    plt.contourf(predicted_2D_var_xplus[i, :, :])
    plt.title("Velocity u + MSE")
    plt.colorbar()
    plt.axis("scaled")
    plt.suptitle("Variance velocity u at time step %s" %predtime)
    plt.tight_layout()
    plt.subplots_adjust(top=0.8)

In [ ]:
for i, predtime in enumerate(prediction_times):
    fig = plt.figure(figsize=(8, 3), dpi=200)
    fig.add_subplot(1,2,1)
    plt.contourf(predicted_2D_var_yminus[i, :, :])
    plt.colorbar()
    plt.axis("scaled")
    plt.title("Velocity v - MSE")
    fig.add_subplot(1,2,2)
    plt.contourf(predicted_2D_var_yplus[i, :, :])
    plt.title("Velocity v + MSE")
    plt.colorbar()
    plt.axis("scaled")
    plt.suptitle("Variance velocity v at time step %s" %predtime)
    plt.tight_layout()
    plt.subplots_adjust(top=0.8)

#### Save data

In [ ]:
prediction_times2 = [alltimes[-1] + 25, alltimes[-1] + 50, alltimes[-1] + 75]
pd.DataFrame(predicted_x,index=prediction_times2).to_csv("velocity_x_prediction.csv")
pd.DataFrame(predicted_y,index=prediction_times2).to_csv("velocity_y_prediction.csv")

In [ ]:
pd.read_csv("velocity_y_prediction_varminus.csv")